In [2]:
import selenium #셀레니움
import pandas as pd #csv를 읽고 dataframe을 사용하기 위한 pandas
from selenium import webdriver #브라우저를 띄우고 컨트롤하기 위한 webdriver
from selenium.webdriver.common.keys import Keys #브라우저에 키입력 용
from selenium.webdriver.common.by import By #webdriver를 이용해 태그를 찾기 위함
from selenium.webdriver.support.ui import WebDriverWait #Explicitly wait을 위함
from webdriver_manager.chrome import ChromeDriverManager #크롬에서 크롤링 진행 크롬 웹 드라이버 설치시 필요
from selenium.webdriver.support import expected_conditions as EC #브라우저에 특정 요소 상태 확인을 위해
from bs4 import BeautifulSoup #브라우저 태그를 가져오고 파싱하기 위함
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException #예외처리를 위한 예외들 

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from webdriver_manager.chrome import ChromeDriverManager

import time
import pyautogui
import pyperclip

# 먼저 chrome_options를 정의합니다.
chrome_options = Options()

# 불필요한 오류 메시지 제거
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

# 브라우저가 종료되지 않도록 설정
chrome_options.add_experimental_option('detach', True)

# ChromeDriverManager를 사용하여 ChromeDriver 설치 경로 설정
service = Service(executable_path=ChromeDriverManager().install())

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome(service = service)
driver.get("https://map.naver.com/v5/")
try:
   element = WebDriverWait(driver, 10).until(
       EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
   ) #입력창이 뜰 때까지 대기
finally:
   pass

keyword = '교촌치킨'
search_box = driver.find_element(By.CLASS_NAME,"input_search")
time.sleep(5)

search_box.send_keys(keyword)
search_box.send_keys(Keys.ENTER)
time.sleep(5)

#첫번째 iframe 안으로 들어가기
driver.switch_to.frame('searchIframe')


#iframe 밖으로 나오기
#driver.switch_to.default_content()

driver.find_element(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]').click()

# 무한 스크롤
lis = driver.find_elements(By.CSS_SELECTOR,'.UEzoS.rTjJo')
before_len = len(lis)

while True:
    #맨 아래로 스크롤하기
    driver.find_element(By.CSS_SELECTOR,'body').send_keys(Keys.END)
    time.sleep(1.5)

    lis = driver.find_elements(By.CSS_SELECTOR,'.UEzoS.rTjJo')
    after_len = len(lis)
    print('스크롤전:', before_len,'스크롤후:',after_len)

    if before_len == after_len:
        break
    before_len = after_len

#모든 지점들 차례로 클릭할 수 있도록 chains에 담기
chains = driver.find_elements(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li')
#chains = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')

#첫 지점부터 차례대로 순회 하면서, 1)리뷰버튼 클릭, 2) 스크롤내려서 더보기 보이게하기,3)더보기 눌러서 반복
                #  4)더보기 안보일 때까지 계속 눌러서 반복 5) 유저네임, 유저가 쓴 리뷰 개수, 평점(있으면), 날짜,     
for chain in chains:
    #지점 클릭
    #search = chain.find_element(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[1]/a')
    search = chain.find_element(By.XPATH, './div[1]/a')
    search.click()
    time.sleep(5)
    #첫번째 Iframe 나가기
    driver.switch_to.default_content()
    #두번쨰 iframe으로 전환
    driver.switch_to.frame(driver.find_element(By.CSS_SELECTOR, 'iframe#entryIframe'))
    #리뷰버튼 클릭
    desired_element = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[4]')
    desired_element.click()

    while True:
        try:
            driver.find_element(By.CSS_SELECTOR,'body').send_keys(Keys.END)
            time.sleep(1.5)
            more_button = driver.find_element(By.CSS_SELECTOR, 'a.fvwqf')
            if more_button.is_displayed():
                more_button.click()
                time.sleep(1)
            else:
                break
        except NoSuchElementException:
            # '더보기' 버튼이 없으면 스크롤을 맨 위로 올림
            driver.find_element(By.CSS_SELECTOR,'body').send_keys(Keys.HOME)
            time.sleep(1.5)
            break
        except StaleElementReferenceException:
            # '더보기' 버튼이 변경되었을 때, 요소를 다시 찾음
            more_button = driver.find_element(By.CSS_SELECTOR, 'a.fvwqf')


    # detailed_soup = BeautifulSoup(driver.page_source, 'html.parser')
    # name = detailed_soup.select_one("h1#_header").text.strip()
    # rating = detailed_soup.select_one("span.PXMot.LXIwF").text.strip().split('점')[1]
    # visited_rev_num = detailed_soup.select(".PXMot")
    # if len(visited_rev_num)>=2:
    #     visited_rev_num = visited_rev_num[1].text

    # print(f"지점 이름: {name}, 평점: {rating}, 방문자리뷰 수 : {visited_rev_num}")

    reviews = driver.find_elements(By.CSS_SELECTOR,'.YeINN')

    for review in reviews:
        name = review.find_element(By.CSS_SELECTOR, '.VYGLG').text.strip()
        date = review.find_element(By.CSS_SELECTOR, '.tzZTd > time').text.strip()

        try:
            # 짧은 리뷰와 긴 리뷰를 구분하여 처리
            if review.find_elements(By.CSS_SELECTOR, '.ZZ4OK.IwhtZ'):
                # 짧은 리뷰 처리
                rev = review.find_element(By.CSS_SELECTOR, '.ZZ4OK.IwhtZ').text.strip()
            elif review.find_elements(By.CSS_SELECTOR, '.ZZ4OK'):
                # 긴 리뷰 처리
                long_review = review.find_element(By.CSS_SELECTOR, '.ZZ4OK')
                try:
                    more_button = long_review.find_element(By.CSS_SELECTOR, 'a > span.rvCSr')
                    more_button.click()
                    time.sleep(1)  # 클릭 후 리뷰가 펼쳐지는 것을 기다림
                except NoSuchElementException:
                    pass  # "더 보기" 버튼이 없으면 패스

                rev = long_review.text.strip()

            # 별점 데이터 처리
            try:
                star_rating_element = review.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li/div[2]')
                star_rating = star_rating_element.text  # 또는 .text 사용
            except NoSuchElementException:
                star_rating = "No rating"

            print(f'User_name: {name}')
            print(f'date: {date}')
            print("Review:", rev)
            print("Star Rating:", star_rating)

        except NoSuchElementException:
            print("Review element not found.")
        except Exception as e:
            print("An error occurred:", e)


스크롤전: 10 스크롤후: 20
스크롤전: 20 스크롤후: 30
스크롤전: 30 스크롤후: 40
스크롤전: 40 스크롤후: 50
스크롤전: 50 스크롤후: 50


NameError: name 'NoSuchElementException' is not defined